### Necessary imports

In [91]:
from PIL import Image
import numpy as np
from collections import namedtuple
from tqdm import tqdm

### Label metadata

In [92]:
#Label meta data from original author's repo
#Credits: https://github.com/mcordts/cityscapesScripts/blob/master/cityscapesscripts/helpers/labels.py
Label = namedtuple( 'Label' , [

    'name'        , # The identifier of this label, e.g. 'car', 'person', ... .
                    # We use them to uniquely name a class

    'id'          , # An integer ID that is associated with this label.
                    # The IDs are used to represent the label in ground truth images
                    # An ID of -1 means that this label does not have an ID and thus
                    # is ignored when creating ground truth images (e.g. license plate).
                    # Do not modify these IDs, since exactly these IDs are expected by the
                    # evaluation server.

    'trainId'     , # Feel free to modify these IDs as suitable for your method. Then create
                    # ground truth images with train IDs, using the tools provided in the
                    # 'preparation' folder. However, make sure to validate or submit results
                    # to our evaluation server using the regular IDs above!
                    # For trainIds, multiple labels might have the same ID. Then, these labels
                    # are mapped to the same class in the ground truth images. For the inverse
                    # mapping, we use the label that is defined first in the list below.
                    # For example, mapping all void-type classes to the same ID in training,
                    # might make sense for some approaches.
                    # Max value is 255!

    'category'    , # The name of the category that this label belongs to

    'categoryId'  , # The ID of this category. Used to create ground truth images
                    # on category level.

    'hasInstances', # Whether this label distinguishes between single instances or not

    'ignoreInEval', # Whether pixels having this class as ground truth label are ignored
                    # during evaluations or not

    'color'       , # The color of this label
    ] )


#--------------------------------------------------------------------------------
# A list of all labels
#--------------------------------------------------------------------------------

# Please adapt the train IDs as appropriate for your approach.
# Note that you might want to ignore labels with ID 255 during training.
# Further note that the current train IDs are only a suggestion. You can use whatever you like.
# Make sure to provide your results using the original IDs and not the training IDs.
# Note that many IDs are ignored in evaluation and thus you never need to predict these!

labels = [
    #       name                     id    trainId   category            catId     hasInstances   ignoreInEval   color
    Label(  'unlabeled'            ,  0 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'ego vehicle'          ,  1 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'rectification border' ,  2 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'out of roi'           ,  3 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'static'               ,  4 ,      255 , 'void'            , 0       , False        , True         , (  0,  0,  0) ),
    Label(  'dynamic'              ,  5 ,      255 , 'void'            , 0       , False        , True         , (111, 74,  0) ),
    Label(  'ground'               ,  6 ,      255 , 'void'            , 0       , False        , True         , ( 81,  0, 81) ),
    Label(  'road'                 ,  7 ,        0 , 'flat'            , 1       , False        , False        , (128, 64,128) ),
    Label(  'sidewalk'             ,  8 ,        1 , 'flat'            , 1       , False        , False        , (244, 35,232) ),
    Label(  'parking'              ,  9 ,      255 , 'flat'            , 1       , False        , True         , (250,170,160) ),
    Label(  'rail track'           , 10 ,      255 , 'flat'            , 1       , False        , True         , (230,150,140) ),
    Label(  'building'             , 11 ,        2 , 'construction'    , 2       , False        , False        , ( 70, 70, 70) ),
    Label(  'wall'                 , 12 ,        3 , 'construction'    , 2       , False        , False        , (102,102,156) ),
    Label(  'fence'                , 13 ,        4 , 'construction'    , 2       , False        , False        , (190,153,153) ),
    Label(  'guard rail'           , 14 ,      255 , 'construction'    , 2       , False        , True         , (180,165,180) ),
    Label(  'bridge'               , 15 ,      255 , 'construction'    , 2       , False        , True         , (150,100,100) ),
    Label(  'tunnel'               , 16 ,      255 , 'construction'    , 2       , False        , True         , (150,120, 90) ),
    Label(  'pole'                 , 17 ,        5 , 'object'          , 3       , False        , False        , (153,153,153) ),
    Label(  'polegroup'            , 18 ,      255 , 'object'          , 3       , False        , True         , (153,153,153) ),
    Label(  'traffic light'        , 19 ,        6 , 'object'          , 3       , False        , False        , (250,170, 30) ),
    Label(  'traffic sign'         , 20 ,        7 , 'object'          , 3       , False        , False        , (220,220,  0) ),
    Label(  'vegetation'           , 21 ,        8 , 'nature'          , 4       , False        , False        , (107,142, 35) ),
    Label(  'terrain'              , 22 ,        9 , 'nature'          , 4       , False        , False        , (152,251,152) ),
    Label(  'sky'                  , 23 ,       10 , 'sky'             , 5       , False        , False        , ( 70,130,180) ),
    Label(  'person'               , 24 ,       11 , 'human'           , 6       , True         , False        , (220, 20, 60) ),
    Label(  'rider'                , 25 ,       12 , 'human'           , 6       , True         , False        , (255,  0,  0) ),
    Label(  'car'                  , 26 ,       13 , 'vehicle'         , 7       , True         , False        , (  0,  0,142) ),
    Label(  'truck'                , 27 ,       14 , 'vehicle'         , 7       , True         , False        , (  0,  0, 70) ),
    Label(  'bus'                  , 28 ,       15 , 'vehicle'         , 7       , True         , False        , (  0, 60,100) ),
    Label(  'caravan'              , 29 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0, 90) ),
    Label(  'trailer'              , 30 ,      255 , 'vehicle'         , 7       , True         , True         , (  0,  0,110) ),
    Label(  'train'                , 31 ,       16 , 'vehicle'         , 7       , True         , False        , (  0, 80,100) ),
    Label(  'motorcycle'           , 32 ,       17 , 'vehicle'         , 7       , True         , False        , (  0,  0,230) ),
    Label(  'bicycle'              , 33 ,       18 , 'vehicle'         , 7       , True         , False        , (119, 11, 32) ),
    Label(  'license plate'        , -1 ,       -1 , 'vehicle'         , 7       , False        , True         , (  0,  0,142) ),
]


#--------------------------------------------------------------------------------
# Create dictionaries for a fast lookup
#--------------------------------------------------------------------------------

# Please refer to the main method below for example usages!

# name to label object
name2label      = { label.name    : label for label in labels           }
# id to label object
id2label        = { label.id      : label for label in labels           }


### List of labels for which one needs the Spatial Prior(SP)

In [93]:
#provide the labels for which the spatial priors are needed
needed_labels = ('road','sidewalk','sky','person','car','truck','motorcycle','bicycle','bridge','traffic light','traffic sign')

In [94]:
needed_name2label = {label.name : label for label in labels if label.name in needed_labels } 
needed_id2label = {label.id : label for label in labels if label.name in needed_labels}


In [95]:
#mapping for label_id to tensor indices
needed_indices_map = {}
for i in range(len(needed_id2label.keys())):
    needed_indices_map[list(needed_id2label.keys())[i]] = i

print(needed_indices_map)

#needed_indices_map can later be used to map : label_name -> label_id ->tensor_index to access appropriate channel

{7: 0, 8: 1, 15: 2, 19: 3, 20: 4, 23: 5, 24: 6, 26: 7, 27: 8, 32: 9, 33: 10}


In [96]:
def get_label_image_path(img_path):
    return img_path.replace("leftImg8bit","gtFine",1)[0:-15]+"gtFine_labelIds.png"

In [97]:
file_pointer = open("./train_imgs_cityscapes_file_list.txt", 'r')
img_path_list = [line.rstrip() for line in file_pointer.readlines()]
# list_of_path_tuples = [(img_path, get_label_image_path(img_path)) for img_path in img_path_list]
label_img_paths = [get_label_image_path(img_path) for img_path in img_path_list]

In [98]:
print(f'There are {len(label_img_paths)} images to process.')

There are 2975 images to process.


In [88]:
hist_tensor = np.zeros((1024, 2048, len(needed_id2label.keys())),dtype=np.float64)
hist_tensor.shape

(1024, 2048, 11)

In [89]:
#read each label image
for label_img_path in tqdm(label_img_paths):
    try:
        im = Image.open(label_img_path)
#         print(im.size)
        temp_np = np.array(im)
#         print(temp_np.shape)
        #for each needed label
        label_id_list = list(needed_indices_map.keys())
        for label_id in label_id_list:
            #process only on the needed labels
            index_val = needed_indices_map[label_id]
            temp_mask = (temp_np == label_id)*1
            
            hist_tensor[:,:,index_val] += temp_mask
            
        
    except IOError as e:
        print(f'{e}')

100%|██████████| 2975/2975 [14:00<00:00,  3.71it/s]


In [90]:
for label_name in list(needed_name2label.keys()):
    label_id = needed_name2label[label_name].id
    index_val = needed_indices_map[label_id]
    temp_im = Image.fromarray(hist_tensor[:,:,index_val])
    temp_im = temp_im.convert('L')
    temp_im.save('./cityscapes_SP/'+label_name+'.png')